# 1.4-agifford-DetermineAnalysisWindowSize
This notebook cycles through the data files in the training set in order to determine the "ideal" window size with which to calcuate the frequency features among labels. Essentially, we want sufficient frequency resolution to extract the whole-number-frequency features we decided on in 1.3, a sufficient number of training samples for model building, and sufficient precision in the estimates of magnitudes at each frequency feature. The trade off will be a smaller window size for more training samples vs. a larger window for better frequency resolution and better estimates of spectral magnitudes.

Since we are working with whole-number frequencies as features, minimally we need 51 data points per sample to compute the spectra (since Fs = 50 Hz).

In [ ]:
import json
import pandas as pd
import plotly.express as px
import numpy as np
import numpy.matlib
from scipy import signal
from scipy.fftpack import fft, fftshift

In [ ]:
with open("../../src/data/train_val_files.json", "r", encoding="utf-8") as infile:
    train_val_files = json.load(infile)
train_files = train_val_files["train"]

with open("../../src/features/frequency_features.json", "r", encoding="utf-8") as infile:
    FEATURES = json.load(infile)

Given that we only want to compute the power for subsets of frequencies (rather than the entire FFT), it MAY be faster to do a direct calculation of just these frequency features, rather than doing a full FFT and then selecting out the frequencies of interest. Apparently, there is an algorithm to do that (the [Goertzel algorithm](https://en.wikipedia.org/wiki/Goertzel_algorithm)), but since I am not familiar with this algorithm I will simply compute the FFT and select the frequencies of interest for the sake of moving the project forward.

Future work can explore whether the Goertzel algorithm would allow for faster feature calculations, which may be necessary for true "real-time" detection of exercise activities.

In [ ]:
# updating functions here to compute the FFT across all measurements in one go rather than
# looping through each

# for this function, it should be sufficient to just calculate the raw FFT rather than
# the normalized one since we only needed the normalized form to identify peaks above a
# threshold
def calculate_frequencies(n_fft, fs=50):
    n_points = 2 * int(np.floor(n_fft / 2))
    if n_fft % 2:
        n_points += 1
    freq = fs/2 * np.linspace(-1, 1, n_points)
    return freq

def calculate_spectrum(ndarray, fs=50):
    X_w = fftshift(fft(WINDOW * ndarray, axis=0), axes=0)
    
    return np.abs(X_w / numpy.matlib.repmat(abs(X_w).max(axis=0), ndarray.shape[0], 1))

def calculate_normed_spectrum(ndarray, fs=50):
    n_fft = df.shape[0]
    window = numpy.matlib.repmat(signal.hann(n_fft), 1, ndarray.shape[1])
    X_w = fft(window * ndarray)
    X_w_norm = 20 * np.log10(
        np.abs(fftshift(X_w / numpy.matlib.repmat(abs(X_w).max(axis=0), ndarray.shape[0], 1)))
    )

    n_points = 2 * int(np.floor(n_fft / 2))
    if n_fft % 2:
        n_points += 1
    freq = fs/2 * np.linspace(-1, 1, n_points)
    return X_w_norm, freq

def get_frequencies_indices(all_freqs, desired_freqs):
    closest_freqs_ix = np.array([(np.abs([af - df for af in all_freqs])).argmin() for df in desired_freqs])
    return closest_freqs_ix

def calculate_windowed_feats(df, freq, cols = None):
    cols = (
        cols or ["accel_x", "accel_y", "accel_z", "gyro_x", "gyro_y", "gyro_z"]
    )
    freq_ixs = [
        get_frequencies_indices(freq, FEATURES[col]) for col in cols
    ]
    feat_cols = [
        col + "_" + str(int(freq_feat)) for col in cols for freq_feat in FEATURES[col]
    ]
    
    results_df = pd.DataFrame(columns=["file_id", "subject_id", "data_id", "window", "t_index", "label", "label_group", *feat_cols])
    for ix, t_start in enumerate(range(0, df.shape[0], N_FFT)):
        if t_start + N_FFT > df.shape[0]-1:
            continue

        df_snip = df.loc[t_start:(t_start + N_FFT - 1), :].reset_index()

        # create new label in case the time period encapsulates parts of multiple activity groups
        if df_snip.label.nunique()>1:
            label = "Transition"
        else:
            label = df_snip.loc[0, "label"]

        if df_snip.label_group.nunique()>1:
            label_group = "Transition"
        else:
            label_group = df_snip.loc[0, "label_group"]

        # compute the spectra
        X_w = calculate_spectrum(df_snip[cols].values)

        # get only the desired frequencies by measurement
        meas_feats = [
            X_w[f_ix, ix] for ix, f_ix in enumerate(freq_ixs)
        ]
        # flatten the features to have same shape as feat_cols
        flat_feats = [
            f_data for col in meas_feats for f_data in col
        ]
        data = {
            "file_id": df.loc[0, "file_id"], 
            "subject_id": df.loc[0, "subject_id"], 
            "data_id": df.loc[0, "data_id"], 
            "window": [N_FFT],
            "t_index": [ix],
            "label": [label],
            "label_group": [label_group],
            **{
                key: [val] for key, val in zip(feat_cols, flat_feats)
            }
        }
        window_df = pd.DataFrame(data=data)
        results_df = pd.concat([results_df, window_df], ignore_index=True)

    return results_df

def agg_statistic(df, agg_col, statistic):
    gb = df.groupby(agg_col, as_index=False).agg({
        col: statistic for col in feats_only.columns[2:]
    })
    gb = gb.rename(columns={agg_col: "activity"})
    gb["statistic"] = statistic
    return gb
        

In [ ]:
FS = 50
n_ffts = [(FS * r) + 1 for r in range(1, 7)] # for 1, 1.5, 2, 2.5, 3-s analysis windows
cols = ["accel_x", "accel_y", "accel_z", "gyro_x", "gyro_y", "gyro_z"]
feat_cols = [
    col + "_" + str(int(freq_feat)) for col in cols for freq_feat in FEATURES[col]
]

grand_results_df = pd.DataFrame(
    columns = [
        "activity",
        *feat_cols,
        "statistic",
        "file",
        "window",
        "count",
        "label_type"
    ]
)
for ix, file in enumerate(train_files):
    print(f"analyzing file {ix+1} of {len(train_files)}...window samples: ", end="")
    df = pd.read_parquet(file, engine="fastparquet")
    for N_FFT in n_ffts:
        print(f"{N_FFT:03}, ", end="")

        WINDOW = numpy.matlib.repmat(signal.hann(N_FFT), 6, 1).T  # 6 for 6 measurement columns
        freq = calculate_frequencies(N_FFT)
        windowed_df = calculate_windowed_feats(df, freq)

        # want mean and std of frequency features by label, label_group for each N_FFT
        feats_only = windowed_df.drop(columns=["file_id", "subject_id", "data_id", "window", "t_index"])

        label_mn = agg_statistic(feats_only, "label", "mean")
        label_std = agg_statistic(feats_only, "label", "std")
        group_mn = agg_statistic(feats_only, "label_group", "mean")
        group_std = agg_statistic(feats_only, "label_group", "std")

        count = feats_only.shape[0]
        label_mn["file"], label_std["file"], group_mn["file"], group_std["file"] = (
            file, file, file, file
        )
        label_mn["window"], label_std["window"], group_mn["window"], group_std["window"] = (
            N_FFT, N_FFT, N_FFT, N_FFT
        )
        label_mn["count"], label_std["count"], group_mn["count"], group_std["count"] = (
            count, count, count, count
        )
        label_mn["label_type"], label_std["label_type"], group_mn["label_type"], group_std["label_type"] = (
            "label", "label", "label_group", "label_group"
        )
        windowed_stats_df = pd.concat(
            [label_mn, label_std, group_mn, group_std],
            ignore_index=True
        )
        grand_results_df = pd.concat([grand_results_df, windowed_stats_df], ignore_index=True)

    print("\r", end="")
    print("\r", end="")
    print("\r", end="")


In [ ]:
grand_label_mn = grand_results_df[
    (grand_results_df["label_type"] == "label") &
    (grand_results_df["statistic"] == "mean")
]
grand_label_std = grand_results_df[
    (grand_results_df["label_type"] == "label") &
    (grand_results_df["statistic"] == "std")
]

grand_group_mn = grand_results_df[
    (grand_results_df["label_type"] == "label_group") &
    (grand_results_df["statistic"] == "mean")
]
grand_group_std = grand_results_df[
    (grand_results_df["label_type"] == "label_group") &
    (grand_results_df["statistic"] == "std")
]

Looks like we can get a minimum of 542,000 samples if we use a 3-second time bin for analyzing frequency features. Even this "minimal" amount should be enough for modeling. But more data is better, so let's check what the precision of data features are across data points to see whether a shorter time window provides sufficient precision,

In [ ]:
grand_label_mn.groupby("window").agg(NSamples=("count", "sum"))

Seems like the grand mean powers for all frequencies across window sizes is fairly consistent, with stds generally less than 0.1. So far so good for all window sizes in estimating mean power.

In [ ]:
(grand_label_mn.groupby("window").agg({
    col: "mean" for col in grand_label_mn.columns[1:66]
}).std()>0.1).sum()

Seems that most often, the 3-second time window had the smallest std of the mean values, followed (curiously) by the 1-second std.

In [ ]:
grand_label_mn.groupby("window").agg({
    col: "std" for col in grand_label_mn.columns[1:66]
}).idxmin(axis=0).reset_index().groupby(0).agg(Count=(0, "count"))

Additionally, the 3-second analysis window generally had the smallest average std across time bins.

In [ ]:
grand_label_std.groupby("window").agg({
    col: "mean" for col in grand_label_mn.columns[1:66]
}).idxmin(axis=0).reset_index().groupby(0).agg(Count=(0, "count"))

Similar results apply at the level of `label_group`. Thus, in terms of estimated accuracy and precision of power estimates, the 3-second time window seems the best. This is what I will use moving forward.

In [ ]:
(grand_group_mn.groupby("window").agg({
    col: "mean" for col in grand_group_mn.columns[1:66]
}).std()>0.1).sum()

In [ ]:
grand_group_mn.groupby("window").agg({
    col: "std" for col in grand_group_mn.columns[1:66]
}).idxmin(axis=0).reset_index().groupby(0).agg(Count=(0, "count"))

In [ ]:
grand_group_std.groupby("window").agg({
    col: "mean" for col in grand_group_mn.columns[1:66]
}).idxmin(axis=0).reset_index().groupby(0).agg(Count=(0, "count"))

**NOTE:** There are definitely more sophisticated ways to assess the appropriate time window in which to analyze the data to detect exercises, in addition to other considerations (e.g., how real-time the classification should be). For the sake of simplicity and moving the project forward, I am leaving more detailed and sohpisticated methods for future versions.